<a href="https://colab.research.google.com/github/ulsals/PengenalanPola/blob/main/UTSPola_Aulia_Salsabella_09021182328016.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UTS Pengenalan Pola

Nama : Aulia Salsabella

NIM  : 09021182328016

In [1]:
from google.colab import drive
import cv2
import math

In [7]:
drive.mount('/content/gdrive')

!cp -r /content/gdrive/MyDrive/Pengenalan_Pola/Gambar_UTS /content

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [8]:
def transpose(matrix):
    """
    Fungsi untuk melakukan transpose matriks.
    """
    # Mengambil dimensi matriks asli
    rows = len(matrix)
    cols = len(matrix[0])
    # Membuat matriks baru (kosong) dengan dimensi yang sudah ditukar
    transposed = [[0 for _ in range(rows)] for _ in range(cols)]
    for i in range(rows):
        for j in range(cols):
            transposed[j][i] = matrix[i][j]
    return transposed

In [9]:
def multiply_matrices(A, B):
    """
    Fungsi untuk melakukan perkalian antara dua matriks (A * B).
    """
    # Mengambil dimensi kedua matriks
    rows_A = len(A)
    cols_A = len(A[0])
    rows_B = len(B)
    cols_B = len(B[0])

    # Syarat perkalian matriks: jumlah kolom matriks A harus sama dengan jumlah baris matriks B.
    if cols_A != rows_B:
        raise ValueError("Ukuran matriks tidak kompatibel untuk perkalian")

    # Membuat matriks hasil yang diisi dengan nol
    result = [[0 for _ in range(cols_B)] for _ in range(rows_A)]
    for i in range(rows_A):
        for j in range(cols_B):
            for k in range(cols_A):
                result[i][j] += A[i][k] * B[k][j]
    return result

In [10]:
def mean_vector(matrix):
    """
    Fungsi untuk menghitung vektor rata-rata dari setiap kolom matriks.
    Langkah penting pertama dalam PCA.
    """
    rows = len(matrix)
    cols = len(matrix[0])
    # Membuat list kosong untuk menampung nilai rata-rata setiap kolom
    mean_vec = [0] * cols
    for j in range(cols):
        col_sum = 0
        # Loop untuk setiap baris di dalam kolom tersebut
        for i in range(rows):
            col_sum += matrix[i][j]
        # Hitung rata-rata dan simpan
        mean_vec[j] = col_sum / rows
    return mean_vec

In [11]:
def subtract_mean(matrix, mean_vec):
    """
    Fungsi untuk melakukan 'mean centering'.
    Setiap elemen di matriks dikurangi dengan nilai rata-rata kolomnya.
    """
    rows = len(matrix)
    cols = len(matrix[0])
    # Membuat matriks baru untuk data yang sudah di-center
    centered_matrix = [[0 for _ in range(cols)] for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            centered_matrix[i][j] = matrix[i][j] - mean_vec[j]
    return centered_matrix

In [12]:
def power_iteration(matrix, iterations=100):
    """
    Fungsi untuk mencari eigenvector dominan.
    """
    rows = len(matrix)
    # mulai dari vektor acak (baris 1)
    b_k = [1.0 for _ in range(rows)]

    # iterasi
    for _ in range(iterations):
        # Kalikan matriks dengan vektor: Ax = A * b_k
        Ax_b_k = [sum(matrix[i][j] * b_k[j] for j in range(rows)) for i in range(rows)]

        # Hitung magnitudo (panjang) dari vektor hasil
        magnitude = math.sqrt(sum(val**2 for val in Ax_b_k))

        # Normalisasi vektor
        if magnitude == 0: return [0]*rows # Menghindari pembagian dengan nol
        b_k = [val / magnitude for val in Ax_b_k]

    return b_k

In [19]:
def get_pca_feature(image_paths):
    """
    Fungsi utama untuk menghitung fitur PCA dari sekumpulan gambar.
    """
    # Muat semua gambar
    TARGET_SIZE = (32, 32)

    data_matrix = []
    print("Membaca, mengubah ukuran, dan meratakan gambar...")
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Gagal memuat gambar: {path}")
            continue

        # Ubah ukuran gambar ke TARGET_SIZE
        img_resized = cv2.resize(img, TARGET_SIZE)

        # Gunakan img_resized untuk proses selanjutnya
        flat_vector = [pixel / 255.0 for row in img_resized for pixel in row]
        data_matrix.append(flat_vector)

    if not data_matrix:
        print("Tidak ada data gambar yang berhasil diproses.")
        return None

    print("Menghitung rata-rata dan sentralisasi data...")
    mean_vec = mean_vector(data_matrix)
    centered_data = subtract_mean(data_matrix, mean_vec)
    print("Menghitung matriks kovarians...")
    centered_data_T = transpose(centered_data)
    covariance_matrix = multiply_matrices(centered_data_T, centered_data)
    print("Mencari Principal Component pertama (Eigenvector)...")
    principal_component = power_iteration(covariance_matrix)
    print("Memproyeksikan data untuk mendapatkan fitur PCA...")
    pc_matrix = [[val] for val in principal_component]
    pca_features = multiply_matrices(centered_data, pc_matrix)
    return pca_features

In [14]:
def get_svd_feature(image_path):
    """
    Fungsi utama untuk menghitung fitur SVD  dari sebuah gambar.
    """
    # Gambar
    i# Muat semua gambar
    TARGET_SIZE = (32, 32)

    data_matrix = []
    print("Membaca, mengubah ukuran, dan meratakan gambar...")
    for path in image_paths:
        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Gagal memuat gambar: {path}")
            continue

        # Ubah ukuran gambar ke TARGET_SIZE
        img_resized = cv2.resize(img, TARGET_SIZE)

        # Gunakan img_resized untuk proses selanjutnya
        flat_vector = [pixel / 255.0 for row in img_resized for pixel in row]
        data_matrix.append(flat_vector)

    if not data_matrix:
        print("Tidak ada data gambar yang berhasil diproses.")
        return None

    # Hitung matriks A_transpose * A dan eigenvalue dari A^T * A.
    A_T = transpose(A)
    AtA = multiply_matrices(A_T, A)

    # Cari eigenvector dominan dari matriks A^T * A.
    eigenvector = power_iteration(AtA)

    # Hitung eigenvalue yang bersesuaian dengan eigenvector tersebut.
    # Rumus: lambda = (Ax * x) / (x * x).
    # Jadi, lambda = Ax * x (dot product).
    Ax = [sum(AtA[i][j] * eigenvector[j] for j in range(len(AtA))) for i in range(len(AtA))]
    eigenvalue = sum(Ax[i] * eigenvector[i] for i in range(len(eigenvector)))

    # Singular value (sigma) terbesar adalah akar kuadrat dari eigenvalue terbesar.
    singular_value = math.sqrt(abs(eigenvalue)) # abs() dipakai untuk menghindari error math domain

    return singular_value

In [17]:
LOCAL_FOLDER_PATH = "/content/Gambar_UTS/"

filenames = ["Lena512warna.bmp", "baboon24.bmp", "bird.bmp","girl-warna.bmp","peppers512warna.bmp"]

# Menggabungkan path folder dengan nama file untuk membuat path lengkap
image_full_paths = [f"{LOCAL_FOLDER_PATH}{fname}" for fname in filenames]

In [20]:
# --- Menjalankan Analisis PCA ---
print("="*30)
print("MENGHITUNG FITUR PCA")
print("="*30)
pca_features = get_pca_feature(image_full_paths)

print("\n--- HASIL FITUR PCA ---")
for i, path in enumerate(filenames):
    print(f"Gambar '{path}': Fitur PCA = {pca_features[i][0]:.4f}")

MENGHITUNG FITUR PCA
Membaca, mengubah ukuran, dan meratakan gambar...
Menghitung rata-rata dan sentralisasi data...
Menghitung matriks kovarians...
Mencari Principal Component pertama (Eigenvector)...
Memproyeksikan data untuk mendapatkan fitur PCA...

--- HASIL FITUR PCA ---
Gambar 'Lena512warna.bmp': Fitur PCA = -0.1024
Gambar 'baboon24.bmp': Fitur PCA = -0.7021
Gambar 'bird.bmp': Fitur PCA = -0.9345
Gambar 'girl-warna.bmp': Fitur PCA = 5.9487
Gambar 'peppers512warna.bmp': Fitur PCA = -4.2096


In [21]:
# --- Menjalankan Analisis SVD ---
print("\n" + "="*30)
print("MENGHITUNG FITUR SVD")
print("="*30)
print("\n--- HASIL FITUR SVD  ---")
for path in image_full_paths:
    svd_feature = get_svd_feature(path)
    filename = path.split('/')[-1]
    print(f"Gambar '{filename}': Fitur SVD = {svd_feature:.4f}")


MENGHITUNG FITUR SVD

--- HASIL FITUR SVD  ---
Gambar 'Lena512warna.bmp': Fitur SVD = 253.8671
Gambar 'baboon24.bmp': Fitur SVD = 263.1416
Gambar 'bird.bmp': Fitur SVD = 130.2107
Gambar 'girl-warna.bmp': Fitur SVD = 165.3048
Gambar 'peppers512warna.bmp': Fitur SVD = 238.9472
